In [1]:
import librosa

In [43]:
from scipy import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [3]:
voices = io.loadmat("./raw.mat")

In [4]:
voices.keys()

dict_keys(['actual_length', 'intensity_label', '__globals__', 'emotion_label', 'actor_label', 'sample_rate', 'modality_label', 'vocal_channel_label', 'feature_matrix', 'statement_label', '__header__', 'repetition_label', '__version__'])

In [7]:
voices["emotion_label"].shape

(1, 1440)

In [8]:
voices["actor_label"].shape

(1, 1440)

In [11]:
voices["feature_matrix"].shape

(1440, 253053)

In [12]:
voices

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Sat Nov 17 15:57:13 2018',
 '__version__': '1.0',
 'actor_label': array([[ 1,  1,  1, ..., 24, 24, 24]], dtype=int32),
 'actual_length': array([[158558, 160160, 156956, ..., 190590, 176176, 174574]], dtype=int32),
 'emotion_label': array([[1, 1, 1, ..., 8, 8, 8]], dtype=int32),
 'feature_matrix': array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00, -3.0517578e-05, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 3.0517578e-05,  3.0517578e-05,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       

In [20]:
(voices["sample_rate"] == 48000).all()

True

In [33]:
x = np.array([[1,2, 3,4], [3,4, 5,6]])

In [38]:
x.reshape(-1, 2,2).sum(axis=2)

array([[ 3,  7],
       [ 7, 11]])

In [28]:
253053//48000

5

In [52]:
features = voices["feature_matrix"]
features = features[:, :253053//48000*48000].reshape(features.shape[0], -1, 24000)

In [53]:
features = np.abs(features)

In [54]:
sliced_mean = features.mean(axis=2)

In [55]:
sliced_max = features.max(axis=2)

In [56]:
sliced_min = features.min(axis=2)

In [57]:
sliced_std = features.std(axis=2)

/opt/conda/lib/python3.5/site-packages/numpy/core/_methods.py:138: RuntimeWarning: invalid value encountered in sqrt
  ret = um.sqrt(ret, out=ret)


In [58]:
sliced_mean.shape

(1440, 10)

In [59]:
sliced_max.shape

(1440, 10)

In [61]:
sliced_std.shape

(1440, 10)

In [85]:
gender = voices["actor_label"] % 2

(1, 1440)

In [87]:
sliced_features = np.hstack((sliced_mean, sliced_max, sliced_min, sliced_std, gender.T))

In [88]:
sliced_features.shape

(1440, 41)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(
    sliced_features, voices["emotion_label"].T, test_size=0.2, random_state=42)


In [90]:
y_train.shape

(1152, 1)

In [91]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train.ravel())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [92]:
accuracy_score(model.predict(X_test), y_test)

0.4791666666666667

### Improve

In [19]:
mfcc = librosa.feature.mfcc(y=voices["feature_matrix"][0, :], sr=48000, dct_type=2)

In [20]:
mfcc.shape

(20, 495)

In [21]:
voices["feature_matrix"].shape

(1440, 253053)

In [35]:
def get_mfccs_one(df):
    mfccs = []
    for i in range(df["feature_matrix"].shape[0]):
        mfccs.append(librosa.feature.mfcc(y=df["feature_matrix"][i, :], sr=48000, dct_type=2).reshape(1, -1))
    
    return np.vstack(mfccs)

In [36]:
mfccs_one = get_mfccs_one(voices)

In [37]:
mfccs_one.shape

(1440, 9900)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(
    mfccs_one, voices["emotion_label"].T, test_size=0.2, random_state=42)

In [44]:
%%time
#model = GradientBoostingClassifier(n_estimators=5, max_depth=100)
model = XGBClassifier(n_jobs=-1)

model.fit(X_train, y_train.ravel())

CPU times: user 27min 17s, sys: 4 s, total: 27min 21s
Wall time: 27min 25s


In [45]:
accuracy_score(model.predict(X_test), y_test)

/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5381944444444444

In [50]:
%%time
#model = GradientBoostingClassifier(n_estimators=5, max_depth=100)
model = XGBClassifier(n_jobs=-1, max_depth=5, n_estimators=150)

model.fit(X_train, y_train.ravel())

CPU times: user 48min 36s, sys: 1.62 s, total: 48min 37s
Wall time: 48min 38s


In [51]:
accuracy_score(model.predict(X_test), y_test)

/opt/conda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5798611111111112